# Challenge 1

## AWS CLI Configuration

In [1]:
from dotenv import load_dotenv
from IPython.core.display import HTML
from IPython.utils.text import SList
import json
import requests

load_dotenv()

True

Use the [sample.env](./sample.env) file to create a `.env` file with the appropriate credentials

In [2]:
%more ./sample.env

AWS_ACCESS_KEY_ID=<AccessKeyId>
AWS_SECRET_ACCESS_KEY=<SecretAccessKey>
AWS_SESSION_TOKEN=<SessionToken>

## CloudFormation stack

### The CloudFormation Template

In [3]:
%more challenge1.yml

AWSTemplateFormatVersion: 2010-09-09
Parameters:
  AMIParameter:
    Type: AWS::EC2::Image::Id
    Default: ami-0b5eea76982371e91
    Description: The operating system ID to use
  SubnetIdParameter:
    Type: AWS::EC2::Subnet::Id
    Description: The public subnet ID inside the VPC to be used with the EC2 Instance
  VpcIdParameter:
    Type: AWS::EC2::VPC::Id
    Description: Enter the VPC id to use in the Security Group and EC2 Instance
Resources:
  myWebAccessSecurityGroup:
    Type: AWS::EC2::SecurityGroup
    Properties:
      GroupDescription: Udacity Challenge 1 security group
      VpcId: !Ref VpcIdParameter
      SecurityGroupIngress:
        - IpProtocol: tcp
          ToPort: 80
          FromPort: 80
          CidrIp: 0.0.0.0/0
      SecurityGroupEgress:
        - IpProtocol: -1
          CidrIp: 0.0.0.0/0
  myWebServerInstance:
    Type: AWS::EC2::Instance
    Properties:
      ImageId: !Ref AMIParameter
      InstanceType: t3.micro
      NetworkInterfaces:
        - Device

### Validate The CloudFormation Template

In [4]:
%%!
export AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID
export AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY
export AWS_SESSION_TOKEN=$AWS_SESSION_TOKEN
aws cloudformation validate-template \
  --template-body file://challenge1.yml

['{',
 '    "Parameters": [',
 '        {',
 '            "ParameterKey": "SubnetIdParameter",',
 '            "NoEcho": false,',
 '            "Description": "The public subnet ID inside the VPC to be used with the EC2 Instance"',
 '        },',
 '        {',
 '            "ParameterKey": "AMIParameter",',
 '            "DefaultValue": "ami-0b5eea76982371e91",',
 '            "NoEcho": false,',
 '            "Description": "The operating system ID to use"',
 '        },',
 '        {',
 '            "ParameterKey": "VpcIdParameter",',
 '            "NoEcho": false,',
 '            "Description": "Enter the VPC id to use in the Security Group and EC2 Instance"',
 '        }',
 '    ]',
 '}']

### Create the CloudFormation Stack

#### Required Parameters

##### VPC ID

In [5]:
vpcs: SList = !export AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID; \
  export AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY; \
  export AWS_SESSION_TOKEN=$AWS_SESSION_TOKEN; \
  aws ec2 describe-vpcs
vpcs

['{',
 '    "Vpcs": [',
 '        {',
 '            "CidrBlock": "172.31.0.0/16",',
 '            "DhcpOptionsId": "dopt-0c30672d3336ade62",',
 '            "State": "available",',
 '            "VpcId": "vpc-083d17b11dfa61ca1",',
 '            "OwnerId": "207979166659",',
 '            "InstanceTenancy": "default",',
 '            "CidrBlockAssociationSet": [',
 '                {',
 '                    "AssociationId": "vpc-cidr-assoc-0b2de702337a3c4bc",',
 '                    "CidrBlock": "172.31.0.0/16",',
 '                    "CidrBlockState": {',
 '                        "State": "associated"',
 '                    }',
 '                }',
 '            ],',
 '            "IsDefault": true',
 '        }',
 '    ]',
 '}']

In [6]:
vpcs_dict: dict = json.loads(vpcs.nlstr)
vpc_id: str = vpcs_dict["Vpcs"][0]["VpcId"]
vpc_id

'vpc-083d17b11dfa61ca1'

##### Subnet ID

In [7]:
subnet_filter: str = f"Name=vpc-id,Values={vpc_id}"
subnets: SList = !export AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID; \
  export AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY; \
  export AWS_SESSION_TOKEN=$AWS_SESSION_TOKEN; \
  aws ec2 describe-subnets \
    --filters $subnet_filter
subnets

['{',
 '    "Subnets": [',
 '        {',
 '            "AvailabilityZone": "us-east-1e",',
 '            "AvailabilityZoneId": "use1-az3",',
 '            "AvailableIpAddressCount": 4091,',
 '            "CidrBlock": "172.31.48.0/20",',
 '            "DefaultForAz": true,',
 '            "MapPublicIpOnLaunch": true,',
 '            "MapCustomerOwnedIpOnLaunch": false,',
 '            "State": "available",',
 '            "SubnetId": "subnet-028fe24cb9a435642",',
 '            "VpcId": "vpc-083d17b11dfa61ca1",',
 '            "OwnerId": "207979166659",',
 '            "AssignIpv6AddressOnCreation": false,',
 '            "Ipv6CidrBlockAssociationSet": [],',
 '            "SubnetArn": "arn:aws:ec2:us-east-1:207979166659:subnet/subnet-028fe24cb9a435642",',
 '            "EnableDns64": false,',
 '            "Ipv6Native": false,',
 '            "PrivateDnsNameOptionsOnLaunch": {',
 '                "HostnameType": "ip-name",',
 '                "EnableResourceNameDnsARecord": false,',
 '  

In [8]:
subnets_dict: dict = json.loads(subnets.nlstr)

for subnet in subnets_dict["Subnets"]:
  # Avoid 0 (us-east-1e since it doesn't have t3.micro)
  if subnet["AvailabilityZone"] != "us-east-1e":
    subnet_id: str = subnet["SubnetId"]
    break
subnet_id

'subnet-0f2528112f1779689'

#### Create Stack

In [9]:
create_stack_command: str = f"""
  export AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID;\
  export AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY;\
  export AWS_SESSION_TOKEN=$AWS_SESSION_TOKEN;\
  aws cloudformation create-stack --stack-name udacitychallengestack \
    --template-body file://challenge1.yml \
    --parameters ParameterKey=SubnetIdParameter,ParameterValue={subnet_id} ParameterKey=VpcIdParameter,ParameterValue={vpc_id}
  """
!{create_stack_command}

{
    "StackId": "arn:aws:cloudformation:us-east-1:207979166659:stack/udacitychallengestack/60e622a0-8ef5-11ed-8871-0eeac476d8ed"
}


### View the CloudFormation Stack

#### Describe the CloudFormation Stack

In [10]:
stacks: SList = !export AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID; \
  export AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY; \
  export AWS_SESSION_TOKEN=$AWS_SESSION_TOKEN; \
  aws cloudformation describe-stacks --stack-name udacitychallengestack
stacks

['{',
 '    "Stacks": [',
 '        {',
 '            "StackId": "arn:aws:cloudformation:us-east-1:207979166659:stack/udacitychallengestack/60e622a0-8ef5-11ed-8871-0eeac476d8ed",',
 '            "StackName": "udacitychallengestack",',
 '            "Parameters": [',
 '                {',
 '                    "ParameterKey": "SubnetIdParameter",',
 '                    "ParameterValue": "subnet-0f2528112f1779689"',
 '                },',
 '                {',
 '                    "ParameterKey": "AMIParameter",',
 '                    "ParameterValue": "ami-0b5eea76982371e91"',
 '                },',
 '                {',
 '                    "ParameterKey": "VpcIdParameter",',
 '                    "ParameterValue": "vpc-083d17b11dfa61ca1"',
 '                }',
 '            ],',
 '            "CreationTime": "2023-01-08T01:40:06.389000+00:00",',
 '            "RollbackConfiguration": {},',
 '            "StackStatus": "CREATE_COMPLETE",',
 '            "DisableRollback": false,',

#### Get the PublicUrl

In [11]:
stacks_dict: dict = json.loads(stacks.nlstr)
outputs: list = stacks_dict["Stacks"][0]["Outputs"]
outputs

[{'OutputKey': 'PublicIPUrl',
  'OutputValue': 'http://35.172.240.26',
  'Description': 'The public ip address to access the data'}]

In [12]:
public_url: str = outputs[0]["OutputValue"]
public_url

'http://35.172.240.26'

#### Navigate To The Public URl

In [13]:
display(
  HTML(
    requests.get(public_url).text
  )
)

## Clean up

In [15]:
%%!
export AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID
export AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY
export AWS_SESSION_TOKEN=$AWS_SESSION_TOKEN
aws cloudformation delete-stack --stack-name udacitychallengestack

[]